In [1]:
import pandas as pd 
import os 


path = "/Users/winson/Developer/scrape_bank"
file_ext = "_Credit.csv"

banks = ['BOA', 'CO', 'Chase', 'Citi', 'Discover']




In [2]:
df_credit = pd.DataFrame(columns=['Transaction Date', 'Description', 'Amount', 'Category', 'Bank'])

# display(df_credit)
for f in banks: 
	data = pd.read_csv(os.path.join(path, f+file_ext))
	# display(data)
	temp_df = pd.DataFrame(columns=['Transaction Date', 'Description', 'Amount', 'Category', 'Bank'])
	if 'Transaction Date' in data.columns: 
		temp_df['Transaction Date'] = data['Transaction Date'] 
	elif 'Date' in data.columns:
		temp_df['Transaction Date'] = data['Date'] 
	elif 'Trans. Date' in data.columns: 
		temp_df['Transaction Date'] = data['Trans. Date']
	else: 
		temp_df['Transaction Date'] = data['Posted Date']
	

	temp_df['Description'] = data['Description'] if 'Description' in data.columns else data['Payee']
	temp_df['Amount'] = data['Amount'] if 'Amount' in data.columns else data['Debit'] 
	if 'Category' in data.columns:
		temp_df['Category'] = data['Category']
	elif 'Type' in data.columns:
		temp_df['Category'] = data['Type'] 
	else: 
		temp_df['Category'] = None

	temp_df['Bank'] = f

	


	# display(temp_df)
	df_credit = pd.concat([df_credit, temp_df], ignore_index=True)


df_credit.sort_values(by=['Transaction Date'], inplace=True, ignore_index=True, ascending=False)

# df_credit['Date'] = pd.to_datetime(df_credit['Date'], format='%m/%d/%Y')
# # df_credit['Post Date'] = pd.to_datetime(df_credit['Post Date'], format='%m/%d/%Y')

# # df_credit.drop(columns=['Memo'], inplace=True)
df_credit['Amount'] = df_credit['Amount'].astype(float).abs()
# # df_credit.reindex(index=df_credit.index[::-1])
display(df_credit)
df_credit.to_csv('Credit.csv')
# df_credit

,Transaction Date,Description,Amount,Category,Bank
0,2024-08-28,COSTCO GAS #0778 FREMONT CA,32.24,None,Citi
1,2024-08-27,WENDY'S 0020 FREMONT CA,8.82,None,Citi
2,2024-08-25,SIZZLING LUNCH SAN JOSE CA,18.05,None,Citi
3,2024-08-24,CHIPOTLE 2045 SAN JOSE CA,11.59,None,Citi
4,2024-08-23,TST*BLAST AND BREW - SAN San Jose CA,92.66,None,Citi
...,...,...,...,...,...
365,2022-08-31,SQ *NICK THE GREEK SANTA,13.66,Food & Drink,Chase
366,2022-08-30,LEMONADE I* LEMONADE I,62.00,Bills & Utilities,Chase
367,2022-08-29,Payment Thank You - Web,24.60,NaN,Chase
368,2022-08-29,PIZZA MY HEART,30.04,Food & Drink,Chase


In [3]:
# df_credit.drop(index=)
df = df_credit
df = df[df['Description'].str.lower().str.contains('payment') == False]
df = df[df['Description'].str.lower().str.contains('pymt') == False]
display(df)


,Transaction Date,Description,Amount,Category,Bank
0,2024-08-28,COSTCO GAS #0778 FREMONT CA,32.24,None,Citi
1,2024-08-27,WENDY'S 0020 FREMONT CA,8.82,None,Citi
2,2024-08-25,SIZZLING LUNCH SAN JOSE CA,18.05,None,Citi
3,2024-08-24,CHIPOTLE 2045 SAN JOSE CA,11.59,None,Citi
4,2024-08-23,TST*BLAST AND BREW - SAN San Jose CA,92.66,None,Citi
...,...,...,...,...,...
364,2022-09-16,MINT MOBILE,264.62,Bills & Utilities,Chase
365,2022-08-31,SQ *NICK THE GREEK SANTA,13.66,Food & Drink,Chase
366,2022-08-30,LEMONADE I* LEMONADE I,62.00,Bills & Utilities,Chase
368,2022-08-29,PIZZA MY HEART,30.04,Food & Drink,Chase


In [87]:
import plotly.express as px  
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def filterByRange(df, start_date, end_date): 	
	filtered_df = df.loc[(df['Transaction Date'] >= start_date) & (df['Transaction Date'] <= end_date)]
	return filtered_df


# def plotHist(df): 
# 	fig = px.histogram(df, x='Date', y='Running Bal.')
# 	fig.update_layout(xaxis=dict(type='category')) 
# 	fig.show() 

# def plotBar(df):

# 	groups = df.groupby('Date')
# 	x = df['Date'].unique() 
# 	y = [groups.get_group(k)['Running Bal.'].iloc[-1] for k, v in groups] 
# 	# for k, v in groups: 
# 	# 	# display(groups.get_group(k))
# 	# 	y.append(groups.get_group(k)['Running Bal.'].iloc[-1])

# 	fig = px.bar( x=x, y=y)
# 	fig.update_layout(xaxis=dict(title='Date', type='category'), yaxis=dict(title='Runnig Bal.')) 
# 	fig.show() 

def plotLine(df): 
	groups = df.groupby('Date')
	x = df['Date'].unique() 
	y = [groups.get_group(k)['Running Bal.'].iloc[-1] for k, v in groups] 	
	
	# fig = px.line(x=x, y=y, render_mode='lines+markers')
	fig = go.Figure()
	fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines+markers',
	))
	fig.update_layout(xaxis=dict(title='Date'), yaxis=dict(title='Runnig Bal.')) 
	fig.show()


def plotPie(df):
	
	s = df.groupby('Category')['Amount'].sum() 
	fig = px.pie(df)
	fig.show()



categories = { 
	'Amazon': 'amazon'
}

def filter(row):
	for k, v in categories.items():  
		print(row)
		if row['Category'] == None: 
			if any(type in row['Description'].lower() for type in v):
				return k
		else: 
			
df['Category'] = df.apply(filter, axis=1)
df



Transaction Date                     2024-08-28
Description         COSTCO GAS #0778 FREMONT CA
Amount                                    32.24
Category                                 Amazon
Bank                                       Citi
Name: 0, dtype: object
Transaction Date                 2024-08-27
Description         WENDY'S 0020 FREMONT CA
Amount                                 8.82
Category                             Amazon
Bank                                   Citi
Name: 1, dtype: object
Transaction Date                    2024-08-25
Description         SIZZLING LUNCH SAN JOSE CA
Amount                                   18.05
Category                                Amazon
Bank                                      Citi
Name: 2, dtype: object
Transaction Date                   2024-08-24
Description         CHIPOTLE 2045 SAN JOSE CA
Amount                                  11.59
Category                               Amazon
Bank                                     Citi
Name: 

,Transaction Date,Description,Amount,Category,Bank
0,2024-08-28,COSTCO GAS #0778 FREMONT CA,32.24,None,Citi
1,2024-08-27,WENDY'S 0020 FREMONT CA,8.82,None,Citi
2,2024-08-25,SIZZLING LUNCH SAN JOSE CA,18.05,None,Citi
3,2024-08-24,CHIPOTLE 2045 SAN JOSE CA,11.59,None,Citi
4,2024-08-23,TST*BLAST AND BREW - SAN San Jose CA,92.66,None,Citi
...,...,...,...,...,...
364,2022-09-16,MINT MOBILE,264.62,Amazon,Chase
365,2022-08-31,SQ *NICK THE GREEK SANTA,13.66,Amazon,Chase
366,2022-08-30,LEMONADE I* LEMONADE I,62.00,Amazon,Chase
368,2022-08-29,PIZZA MY HEART,30.04,Amazon,Chase


In [81]:
df['Category'].unique()

array([None, 'Amazon'], dtype=object)

In [56]:
start_date = '2024-07-01'
end_date = '2024-07-31'
monthly_df = filterByRange(df, start_date, end_date)
display(monthly_df) 
fig = px.pie(monthly_df, names='Category', values='Amount')
fig.show()

,Transaction Date,Description,Amount,Category,Bank
27,2024-07-31,PANDA EXPRESS #2957 P olo.com CA,9.70,None,Citi
28,2024-07-31,McDonalds 13001,5.02,Dining,CO
29,2024-07-30,POTATO CORNER,7.75,Dining,CO
30,2024-07-30,COSTCO GAS #1004 SAN JOSE CA,29.66,None,Citi
31,2024-07-28,MITSUWA MRKTPLACE SJ SAN JOSE CAAPPLE PAY ENDI...,2.23,Supermarkets,Discover
32,2024-07-27,OBAITORI MILPITAS CA,100.00,None,Citi
33,2024-07-27,TEA COUNTRY INC,4.75,Dining,CO
34,2024-07-27,TOFU PLUS. CUPERTINO CA,22.73,None,Citi
35,2024-07-26,SQ *ZERO& TEA AND BOBA,16.81,Dining,CO
37,2024-07-25,TST* PARIS BAGUETTE - SAN,7.98,Dining,CO
